In [3]:
# semas_pdf_crawler_full.py
import re, json, time, random
from pathlib import Path
from urllib.parse import urlparse, parse_qs
from typing import Optional, List

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ===================== 설정 =====================
BASE_URL   = "https://ols.semas.or.kr"
LIST_URL   = "https://ols.semas.or.kr/ols/man/SMAN055M/page.do"  # 공지사항(직접대출) 목록
OUTDIR     = Path("downloads/pdf")
LEDGER     = Path("downloads/ledger.json")
MAX_PAGES  = 200  # 최대 순회 페이지 (안전 상한)

# 상세 페이지의 첨부 링크 선택자 (fnDownFile)
ATTACH_SEL = "#contents div.board_view div.file_down a[onclick^='fnDownFile']"
ALLOWED_EXTS = {".pdf", ".hwp"}  # 필요 시 확장

# 슬립/지터
S_WAIT   = (1.0, 2.0)
S_CLICK  = (0.5, 1.0)
CHECK_INTERVAL  = 1.0
MAX_WAIT_DL_SEC = 120
# =================================================

SKIP_EMPTY_LABELS   = True                 # 라벨이 비어있거나 의미없는 경우 스킵
PLACEHOLDER_LABELS  = {".", "..", "...", "…"}  # 자리표시자 텍스트
MIN_LABEL_LEN       = 2                    # 이 길이 미만 라벨은 무의미로 간주


def nap(a: float, b: float):
    time.sleep(random.uniform(a, b))


def safe_slug(s: str) -> str:
    return re.sub(r"[^0-9A-Za-z가-힣._-]+", "_", s).strip("_")[:160]


def make_driver() -> webdriver.Chrome:
    OUTDIR.mkdir(parents=True, exist_ok=True)
    opts = webdriver.ChromeOptions()
    opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1400,2000")
    prefs = {
        "download.default_directory": str(OUTDIR.resolve()),
        "download.prompt_for_download": False,
        "plugins.always_open_pdf_externally": True,
        "safebrowsing.enabled": True,
        "safebrowsing.disable_download_protection": True,
    }
    opts.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=opts)
    # 헤드리스에서도 다운로드 허용
    driver.execute_cdp_cmd("Page.setDownloadBehavior", {
        "behavior": "allow",
        "downloadPath": str(OUTDIR.resolve())
    })
    return driver


def wait_list_loaded(driver: webdriver.Chrome, want_page: Optional[int] = None):
    """#resultList 내부에 fnGoModNoti 링크가 생길 때까지 대기.
       want_page가 주어지면 페이저의 활성 페이지도 확인."""
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, "#resultList tr a[onclick*='fnGoModNoti']"))
    )
    if want_page:
        def page_ok(drv):
            try:
                # 활성 페이지 표시 a.active 또는 .on 등 대응
                active = (drv.find_element(By.CSS_SELECTOR, ".pagenum_box li a.active")).text.strip()
                return active == str(want_page)
            except Exception:
                return True
        WebDriverWait(driver, 10).until(page_ok)


def get_last_page(driver: webdriver.Chrome) -> int:
    """페이저의 최댓값 추정."""
    try:
        nums = []
        for a in driver.find_elements(By.CSS_SELECTOR, ".pagenum_box li a"):
            t = a.text.strip()
            if t.isdigit():
                nums.append(int(t))
        return max(nums) if nums else 1
    except Exception:
        return 1


def parse_rows(driver: webdriver.Chrome) -> List[dict]:
    """목록의 각 행에서 bltwtrSeq, bbsTypeCd, 제목 파싱."""
    items = []
    rows = driver.find_elements(By.CSS_SELECTOR, "#resultList tr")
    for tr in rows:
        try:
            a = tr.find_element(By.CSS_SELECTOR, "a[onclick*='fnGoModNoti']")
            oc = a.get_attribute("onclick") or ""
            # 형태: fnGoModNoti(299,01) 또는 fnGoModNoti(299,1)
            m = re.search(r"fnGoModNoti\((\d+)\s*,\s*0?(\d+)\)", oc)
            if not m:
                continue
            seq   = m.group(1)
            btype = m.group(2).zfill(2)
            title = (a.text or "").strip()
            items.append({"seq": seq, "btype": btype, "title": title})
        except Exception:
            continue
    return items


def ensure_detail_form_selector(driver: webdriver.Chrome) -> dict:
    """상세로 들어갈 때 사용할 숨은 폼과 필드 선택자 확인."""
    # 기본 값
    sel = {
        "form":    "#notiNums",
        "seq":     "#notiNum",
        "btype":   "#bbsTypeCd",
    }
    # 폼이 없으면 action 으로 추정
    try:
        driver.find_element(By.CSS_SELECTOR, sel["form"])
        return sel
    except Exception:
        pass

    # 대체: action이 SMAN052M/page.do 인 form 찾기
    alts = driver.find_elements(By.CSS_SELECTOR, "form[action*='SMAN052M/page.do']")
    if alts:
        form_id = alts[0].get_attribute("id") or "form[action*='SMAN052M/page.do']"
        sel["form"] = f"#{form_id}" if form_id != "form[action*='SMAN052M/page.do']" else "form[action*='SMAN052M/page.do']"
        # 필드명 추정
        for cand in ("#notiNum", "input[name='notiNum']", "input[name='bltwtrSeq']", "#bltwtrSeq"):
            if driver.find_elements(By.CSS_SELECTOR, cand):
                sel["seq"] = cand
                break
        for cand in ("#bbsTypeCd", "input[name='bbsTypeCd']", "#bbsType"):
            if driver.find_elements(By.CSS_SELECTOR, cand):
                sel["btype"] = cand
                break
    return sel


def open_detail_same_tab(driver: webdriver.Chrome, seq: str, btype: str):
    """숨은 폼을 같은 탭에서 POST 제출하여 상세 진입."""
    sel = ensure_detail_form_selector(driver)
    script = f"""
      (function(){{
        var f = document.querySelector("{sel['form']}");
        if(!f) throw new Error("detail form not found");
        var s = f.querySelector("{sel['seq']}");
        var t = f.querySelector("{sel['btype']}");
        if(!s || !t) throw new Error("detail fields not found");
        s.value = "{seq}";
        t.value = ("{btype}").padStart(2,'0');
        f.removeAttribute('target');  // 같은 탭
        f.submit();
      }})();
    """
    driver.execute_script(script)
    WebDriverWait(driver, 20).until(EC.url_contains("/ols/man/SMAN052M/page.do"))
    WebDriverWait(driver, 20).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "#contents .board_view")
    ))


def list_downloaded_files():
    return {p for p in OUTDIR.iterdir()
            if p.is_file() and not p.name.endswith(".crdownload")}


def wait_for_new_download(before_set):
    waited = 0.0
    while waited < MAX_WAIT_DL_SEC:
        time.sleep(CHECK_INTERVAL)
        waited += CHECK_INTERVAL
        # 진행 중(.crdownload)이 있으면 계속 대기
        if list(OUTDIR.glob("*.crdownload")):
            continue
        after = list_downloaded_files()
        new_files = after - before_set
        if new_files:
            return max(new_files, key=lambda p: p.stat().st_mtime)
    return None

SUPPRESS_VERIFY_FAIL = True   # 확인 실패 메시지 출력 안 함
RECENT_FALLBACK_SEC  = 45     # 확인 실패 시, 최근 생성 파일을 이 초수 내에서 후보로 인정

def pick_recent_file_since(ts: float):
    """클릭 시각(ts) 이후에 생성/갱신된 최신 파일을 추정해서 돌려준다."""
    try:
        candidates = []
        for p in OUTDIR.iterdir():
            if p.is_file() and not p.name.endswith(".crdownload"):
                # 확장자 제한(원하면 주석 처리)
                if ALLOWED_EXTS and p.suffix.lower() not in ALLOWED_EXTS:
                    continue
                if p.stat().st_mtime >= ts - 1:  # 시계 오차 보정
                    candidates.append(p)
        if not candidates:
            return None
        return max(candidates, key=lambda x: x.stat().st_mtime)
    except Exception:
        return None

def download_attachments_on_detail(driver: webdriver.Chrome, seq: str):
    """상세에서 첨부 전체(fnDownFile) 다운로드."""
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "body")))
    nap(*S_WAIT)

    anchors = driver.find_elements(By.CSS_SELECTOR, ATTACH_SEL)
    if not anchors:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        anchors = soup.select(ATTACH_SEL)
        if not anchors:
            print("  [INFO] 첨부 없음")
            return

    print(f"  [INFO] 첨부 {len(anchors)}건 발견")
    for i, a in enumerate(anchors, start=1):
        is_bs4  = not hasattr(a, "get_attribute")
        raw_txt = (a.get_text() if is_bs4 else a.text or "")
        label   = re.sub(r"\s+", " ", raw_txt).strip() if raw_txt else ""

        # onclick에서 index 추출
        onclick = (a.get("onclick") if is_bs4 else a.get_attribute("onclick")) or ""
        m = re.search(r"fnDownFile\((\d+)\)", onclick)
        idx = int(m.group(1)) if m else None

        # title/aria-label 대체 텍스트도 확보
        alt_title = ""
        if is_bs4:
            alt_title = (a.get("title") or a.get("aria-label") or "").strip()
        else:
            alt_title = ((a.get_attribute("title") or "") + " " + (a.get_attribute("aria-label") or "")).strip()

        # === 스킵 조건 ===
        should_skip = False
        if SKIP_EMPTY_LABELS:
            # 1) 라벨이 완전 비어있거나 자리표시자
            if (not label) or (label in PLACEHOLDER_LABELS) or (len(label) < MIN_LABEL_LEN):
                # 대체 텍스트도 없고, idx도 없으면 확실히 스킵
                if not alt_title or idx is None:
                    should_skip = True
        if idx is None:
            # fnDownFile 인덱스를 못 읽으면 스킵
            should_skip = True

        if should_skip:
            print(f"   - ({i}) [SKIP] 빈/무의미 라벨 또는 인덱스 없음 (idx={idx}, label='{label}')")
            continue

        # 라벨이 비어있어도, 나중 리네임을 위해 최소한의 안전 라벨 보정
        if not label and alt_title:
            label = alt_title
        if not label:
            label = f"file_{seq}_{i}"  # 최후의 보정

        print(f"   - ({i}) idx={idx}, label='{label}'")

        # 클릭 전 기준점들
        before   = list_downloaded_files()

        # 실행
        try:
            if is_bs4:
                # bs4 태그면 DOM에서 동일 onclick 인덱스로 다시 찾아 클릭(라벨 미사용)
                target = driver.find_element(By.CSS_SELECTOR, f"a[onclick='fnDownFile({idx})']")
                driver.execute_script("arguments[0].click();", target)
            else:
                # 이미 webelement면 바로 JS 클릭
                driver.execute_script("arguments[0].click();", a)
        except Exception as e:
            print(f"     [ERR] 클릭 실패: {e}")
            continue

        nap(*S_CLICK)

        # 디렉토리 차집합 방식으로 신규 파일 감지
        saved = wait_for_new_download(before)
        if not saved:
            # 조용히 넘어가고 다음 첨부 처리 (원하면 여기서 로그를 더 줄여도 됨)
            print(f"     [SKIP] 다운로드 확인 실패(조용히 패스): '{label}'")
            continue

        # 파일명 정리(가능하면 seq_원문명.확장자)
        try:
            base = label if label else saved.name
            if "." not in base:
                base += saved.suffix
            target = OUTDIR / safe_slug(f"{seq}_{base}")
            if target.exists():
                print(f"     [OK] 저장(중복 존재): {saved.name}")
            else:
                saved.rename(target)
                print(f"     [OK] 저장 완료: {target.name}")
        except Exception as e:
            print(f"     [WARN] 리네임 실패: {e}")

        nap(*S_WAIT)

def main():
    OUTDIR.mkdir(parents=True, exist_ok=True)
    done = set(json.loads(LEDGER.read_text(encoding="utf-8"))) if LEDGER.exists() else set()

    driver = make_driver()
    try:
        # 1) 목록 진입 → 초기 AJAX 로딩 대기
        driver.get(LIST_URL)
        wait_list_loaded(driver)
        last_page = min(get_last_page(driver), MAX_PAGES)
        print(f"[INFO] 총 페이지 추정: {last_page}")

        # 2) 페이지 순회
        for page in range(1, last_page + 1):
            # 페이지 전환은 클릭이 아니라 JS 함수(fnSearch) 호출
            driver.execute_script("fnSearch(arguments[0]);", page)
            wait_list_loaded(driver, want_page=page)
            nap(*S_WAIT)

            items = parse_rows(driver)
            print(f"\n=== 페이지 {page} / 항목 {len(items)}건 ===")

            for it in items:
                seq, btype, title = it["seq"], it["btype"], it["title"]
                if seq in done:
                    continue

                print(f"[DETAIL] seq={seq}, type={btype}, title={title}")
                # 3) 상세 진입(같은 탭으로 POST)
                open_detail_same_tab(driver, seq, btype)

                # 4) 첨부 전체 다운로드
                try:
                    download_attachments_on_detail(driver, seq)
                finally:
                    # 5) 목록으로 복귀
                    driver.back()
                    wait_list_loaded(driver, want_page=page)

                # 6) 기록
                done.add(seq)
                LEDGER.write_text(json.dumps(sorted(done), ensure_ascii=False, indent=2),
                                  encoding="utf-8")

        print("\n작업 완료.")
    finally:
        driver.quit()


if __name__ == "__main__":
    main()


[INFO] 총 페이지 추정: 10

=== 페이지 1 / 항목 10건 ===
[DETAIL] seq=299, type=01, title=「코로나19피해 소상공인 분할상환 특례」제도 접수 안내
  [INFO] 첨부 3건 발견
   - (1) idx=2, label='「코로나19피해 소상공인 분할상환 특례」제도 신규신청 가이드.pdf'
     [OK] 저장 완료: 299__코로나19피해_소상공인_분할상환_특례_제도_신규신청_가이드.pdf
   - (2) idx=3, label='「코로나19피해 소상공인 분할상환 특례」제도 기존제도활용 추가신청 가이드.pdf'
     [OK] 저장 완료: 299__코로나19피해_소상공인_분할상환_특례_제도_기존제도활용_추가신청_가이드.pdf
   - (3) idx=4, label='2025년 코로나19 피해 소상공인 분할상환지원 공고.hwp'
     [OK] 저장 완료: 299_2025년_코로나19_피해_소상공인_분할상환지원_공고.hwp
[DETAIL] seq=274, type=01, title=「정책자금 상환연장」대상 확대 안내
  [INFO] 첨부 3건 발견
   - (1) idx=2, label='정책자금 상환연장_신청매뉴얼.pdf'
     [OK] 저장 완료: 274_정책자금_상환연장_신청매뉴얼.pdf
   - (2) idx=3, label='「정책자금 상환연장」대상 확대 안내.pdf'
     [OK] 저장 완료: 274__정책자금_상환연장_대상_확대_안내.pdf
   - (3) idx=4, label='기업(신용)정보 수집·이용·제공·조회·활용 동의서.hwp'
     [OK] 저장 완료: 274_기업_신용_정보_수집_이용_제공_조회_활용_동의서.hwp
[DETAIL] seq=261, type=01, title=[2025년] 소상공인정책자금(직접대출) 공통 신청서식 게시(수정)
  [INFO] 첨부 2건 발견
   - (1) idx=3, label='소상공인정책자금(직접대출) 공통 신청서식(수정).hwp'
   